In [1]:
val = 1
print('global', val)

def foo():
    val = 2
    print('local', val)
    
    def glob():
        global val
        print('glob', val)
        val += 10
        
    def loc():
        nonlocal val
        print('loc', val)
        val += 20
        
    glob()
    loc()
        
    print('local', val)
    
foo()
    
print('global', val)

global 1
local 2
glob 1
loc 2
local 22
global 11


---

In [2]:
def callback(fn, a, b):
    fn(a, b)

## Global

In [3]:
glob_immutable_idx = None
glob_mutable_results = None

def hook(a, b):
    global glob_immutable_idx, glob_mutable_results
    print(glob_immutable_idx, f'{a}+{b}={a+b}', glob_mutable_results)
    glob_immutable_idx += 1
    glob_mutable_results.append(a+b)

In [4]:
global glob_immutable_idx, glob_mutable_results
glob_immutable_idx = 0
glob_mutable_results = []

for a in range(1, 4):
    for b in range(1, 5):
        callback(hook, a, b)
        if glob_immutable_idx > 9:
            glob_immutable_idx = 0

0 1+1=2 []
1 1+2=3 [2]
2 1+3=4 [2, 3]
3 1+4=5 [2, 3, 4]
4 2+1=3 [2, 3, 4, 5]
5 2+2=4 [2, 3, 4, 5, 3]
6 2+3=5 [2, 3, 4, 5, 3, 4]
7 2+4=6 [2, 3, 4, 5, 3, 4, 5]
8 3+1=4 [2, 3, 4, 5, 3, 4, 5, 6]
9 3+2=5 [2, 3, 4, 5, 3, 4, 5, 6, 4]
0 3+3=6 [2, 3, 4, 5, 3, 4, 5, 6, 4, 5]
1 3+4=7 [2, 3, 4, 5, 3, 4, 5, 6, 4, 5, 6]


## Extended Hook function

In [5]:
def hook(a, b, c=0):
    print(f'{a=}, {b=}, {c=}')

In [6]:
def hook_extend(a, b, d, c=0, e=1): # add args 'd' and kwargs 'e'
    print(f'{a=}, {b=}, {d=}, {c=}, {e=}')

In [7]:
hook(1,2)

a=1, b=2, c=0


In [8]:
hook(1,2,3)

a=1, b=2, c=3


In [9]:
hook_extend(1,2,3)

a=1, b=2, d=3, c=0, e=1


In [10]:
def hook(a, b, *, c=0):
    print(f'{a=}, {b=}, {c=}')

In [11]:
def hook_extend(a, b, d, *, c=0, e=1): # add args 'd' and kwargs 'e'
    print(f'{a=}, {b=}, {d=}, {c=}, {e=}')

In [12]:
hook(1,2)

a=1, b=2, c=0


In [13]:
hook(1,2,3)

TypeError: hook() takes 2 positional arguments but 3 were given

In [14]:
hook_extend(1,2,3)

a=1, b=2, d=3, c=0, e=1


## Decorator

In [15]:
def hook_extend(a, b, immutable_idx=None, mutable_results=None): # mutable_results=[]): # 
    print(immutable_idx, f'{a}+{b}={a+b}', mutable_results)
    immutable_idx += 1
    mutable_results.append(a+b)

In [16]:
def add_extra_kwargs(**extra_kwargs):
    def decorator(fn):
        def inner(*args, **kwargs):
            return fn(*args, **kwargs, **extra_kwargs)
        return inner
    return decorator

In [17]:
idx = 0
results = []
for a in range(1, 4):
    for b in range(1, 5):
        hook = add_extra_kwargs(immutable_idx=idx, mutable_results=results)(hook_extend)
        callback(hook, a, b)
        if idx > 9:
            idx = 0

0 1+1=2 []
0 1+2=3 [2]
0 1+3=4 [2, 3]
0 1+4=5 [2, 3, 4]
0 2+1=3 [2, 3, 4, 5]
0 2+2=4 [2, 3, 4, 5, 3]
0 2+3=5 [2, 3, 4, 5, 3, 4]
0 2+4=6 [2, 3, 4, 5, 3, 4, 5]
0 3+1=4 [2, 3, 4, 5, 3, 4, 5, 6]
0 3+2=5 [2, 3, 4, 5, 3, 4, 5, 6, 4]
0 3+3=6 [2, 3, 4, 5, 3, 4, 5, 6, 4, 5]
0 3+4=7 [2, 3, 4, 5, 3, 4, 5, 6, 4, 5, 6]


In [42]:
class Integer:
    def __init__(self, idx):
        self.__idx = idx
        
    def __iadd__(self, value):
        self.__idx += value
        return self
    
    def __gt__(self, value):
        return self.__idx > value
        
    def __repr__(self):
        return str(self.__idx)

In [43]:
idx = Integer(0)
results = []
for a in range(1, 4):
    for b in range(1, 5):
        hook = add_extra_kwargs(immutable_idx=idx, mutable_results=results)(hook_extend)
        callback(hook, a, b)
        if idx > 9:
            idx = Integer(0)

0 1+1=2 []
1 1+2=3 [2]
2 1+3=4 [2, 3]
3 1+4=5 [2, 3, 4]
4 2+1=3 [2, 3, 4, 5]
5 2+2=4 [2, 3, 4, 5, 3]
6 2+3=5 [2, 3, 4, 5, 3, 4]
7 2+4=6 [2, 3, 4, 5, 3, 4, 5]
8 3+1=4 [2, 3, 4, 5, 3, 4, 5, 6]
9 3+2=5 [2, 3, 4, 5, 3, 4, 5, 6, 4]
0 3+3=6 [2, 3, 4, 5, 3, 4, 5, 6, 4, 5]
1 3+4=7 [2, 3, 4, 5, 3, 4, 5, 6, 4, 5, 6]


## Callable Object

In [21]:
class HookExtend:
    def __init__(self, fn):
        self.__fn = fn
        self.__kwargs = dict()
        
    def update_kwargs(self, **kwargs):
        self.__kwargs.update(kwargs)
        
    def __call__(self, *args, **kwargs):
        return self.__fn(*args, **kwargs, **self.__kwargs)

In [22]:
idx = Integer(0)
results = []
hook = HookExtend(hook_extend)
for a in range(1, 4):
    for b in range(1, 5):
        hook.update_kwargs(immutable_idx=idx, mutable_results=results)
        callback(hook, a, b)
        if idx > 9:
            idx = Integer(0)

0 1+1=2 []
1 1+2=3 [2]
2 1+3=4 [2, 3]
3 1+4=5 [2, 3, 4]
4 2+1=3 [2, 3, 4, 5]
5 2+2=4 [2, 3, 4, 5, 3]
6 2+3=5 [2, 3, 4, 5, 3, 4]
7 2+4=6 [2, 3, 4, 5, 3, 4, 5]
8 3+1=4 [2, 3, 4, 5, 3, 4, 5, 6]
9 3+2=5 [2, 3, 4, 5, 3, 4, 5, 6, 4]
0 3+3=6 [2, 3, 4, 5, 3, 4, 5, 6, 4, 5]
1 3+4=7 [2, 3, 4, 5, 3, 4, 5, 6, 4, 5, 6]


---

In [23]:
import os

import cv2
import imutils

import numpy as np

In [24]:
%load_ext watermark
%watermark -v -m -p cv2,imutils,numpy

Python implementation: CPython
Python version       : 3.9.5
IPython version      : 8.7.0

cv2    : 4.6.0
imutils: 0.5.4
numpy  : 1.23.5

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 69 Stepping 1, GenuineIntel
CPU cores   : 4
Architecture: 64bit



In [25]:
def read_image(file_name, margin = 1000):
    # Open image
    img = cv2.imread(file_name)
    if margin:
        img = cv2.copyMakeBorder(img, margin, margin, margin, margin, borderType=cv2.BORDER_CONSTANT, value=(255, 255, 255))

    return img

def resize_image(img, height=1000):
    # Resize the image
    rsz = imutils.resize(img, height=height)

    # Calculate the ratio
    ratio = img.shape[0] / rsz.shape[0]
    
    return rsz, ratio

def rotate(img, angle, center = None, scale = 1.0):
    (h, w) = img.shape[:2]

    if center is None:
        center = (w / 2, h / 2)

    # Perform the rotation
    M = cv2.getRotationMatrix2D(center, angle, scale)
    rtt = cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_LINEAR, borderValue=(255, 255, 255))

    return rtt

In [26]:
def onMouseExtend(event, x, y, flags, param, img=None, pos_list=None):
    if event == cv2.EVENT_LBUTTONDOWN:
        pos_list.append((x, y))
        cv2.circle(img, pos_list[-1], 5, (0,0,255), -1)
        
class HookExtend:
    def __init__(self, fn):
        self.__fn = fn
        self.__kwargs = dict()
        
    def update_kwargs(self, **kwargs):
        self.__kwargs.update(kwargs)
        
    def __call__(self, *args, **kwargs):
        return self.__fn(*args, **kwargs, **self.__kwargs)

In [37]:
def cv_crop_image(img, label, height=900, threshold=False, verbose=False):
    rsz, ratio = resize_image(img, height=height)
    if threshold:
        rsz = cv2.cvtColor(rsz, cv2.COLOR_BGR2GRAY)
        rsz = cv2.adaptiveThreshold(rsz, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 5, 0)
        rsz = cv2.cvtColor(rsz, cv2.COLOR_GRAY2BGR)

    image = rsz.copy()
    pos_list = []

    onMouse = HookExtend(onMouseExtend)
    onMouse.update_kwargs(img=image, pos_list=pos_list)

    rect_rsz = None
    label = label+' ~ Cropping'
    cv2.namedWindow(label)
    cv2.setMouseCallback(label, onMouse)
    while True:
        cv2.imshow(label, image)
        
        key = cv2.waitKey(20) & 0xFF
        if key == 27: # Escape
            pos_list = []
            image = rsz.copy()
            onMouse.update_kwargs(img=image, pos_list=pos_list)
        elif key in (13, 32): # Enter or Space
            if verbose: print(pos_list)
            rect_rsz = np.asarray(pos_list).reshape([-1, 1, 2])
            break
    cv2.destroyAllWindows()
    
    rect_img = ((rect_rsz.astype('float'))*ratio).astype('int')
    x, y, w, h = cv2.boundingRect(rect_img)
    img = img[y:y+h, x:x+w, :]
    
    return img

def cv_rotate_image(img, label, height=900, threshold=False, verbose=False):
    rsz, ratio = resize_image(img, height=height)
    if threshold:
        rsz = cv2.cvtColor(rsz, cv2.COLOR_BGR2GRAY)
        rsz = cv2.adaptiveThreshold(rsz, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 21 , 2)
        rsz = cv2.cvtColor(rsz, cv2.COLOR_GRAY2BGR)

    image = rsz.copy()
    pos_list = []

    onMouse = HookExtend(onMouseExtend)
    onMouse.update_kwargs(img=image, pos_list=pos_list)

    left, right = None, None
    label = label+' ~ Angle'
    cv2.namedWindow(label)
    cv2.setMouseCallback(label, onMouse)
    while True:
        cv2.imshow(label, image)
        
        key = cv2.waitKey(20) & 0xFF
        if key == 27: # Escape
            pos_list = []
            image=rsz.copy()
            onMouse.update_kwargs(img=image, pos_list=pos_list)
        elif key in (13, 32): # Enter or Space
            if len(pos_list) < 2:
                continue
            if verbose: print(pos_list)
            left, right = pos_list[:2]
            break
    cv2.destroyAllWindows()
    
    angle = np.arctan( (right[1]-left[1])/(right[0]-left[0]) ) * 180 / np.pi
    if verbose: print(angle)
        
    img = rotate(img, angle)
    
    return img

def cv_save_image(img, file_name, label, verbose=False):
    rsz, ratio = resize_image(img, height=800)
    done = False
    
    label = label+' ~ Result'
    cv2.namedWindow(label)

    cv2.imshow(label, rsz)

    key = cv2.waitKey() & 0xFF
    if key == 27: # Escape
        done = False
    elif key in (13, 32): # Enter or Space
        cv2.imwrite(file_name, img)
        done = True
        
    cv2.destroyAllWindows()
    
    return done

In [38]:
def cv2_manual_photo(file_name, iputdir='', oputdir='', label=None):
    oputdir = oputdir or iputdir
    img = read_image(os.path.join(iputdir, file_name), margin=0)

    while True:
        img_proc = cv_crop_image(img, label=label, threshold=True)

        img_proc = cv_rotate_image(img_proc, label=label, threshold=True)

        img_proc = cv_crop_image(img_proc, label=label, threshold=True)
        
        if cv_save_image(img_proc, os.path.join(oputdir, file_name.replace('_orig.png', '.png')), label=label):
            break

In [39]:
iputdir = 'photo_2_crop'

png_name_list = [png for png in os.listdir(iputdir) if png.endswith('_orig.png')]
for i, png_name in enumerate(png_name_list, start=1):
    cv2_manual_photo(png_name, iputdir, label='[{} of {}] {}'.format(i, len(png_name_list), png_name))
        
print("Done!")

Done!
